In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
from time import time
import warnings

In [2]:
df=pd.read_csv('df_final.csv')

In [3]:
df.head()

,customer_unique_id,Recency,Frequency,MonetaryValue,Cluster,order_purchase_timestamp,customer_zip_code_prefix,price,review_score,seller_zip_code_prefix
0,000bfa1d2f1a41876493be685390d6d3,335,2,93.70,2,2017-09-29 14:46:58,11095,35.0,5,15013
1,000bfa1d2f1a41876493be685390d6d3,335,2,93.70,2,2017-09-29 14:46:58,11095,35.0,4,15013
2,001147e649a7b1afd577e873841632dd,364,2,424.32,0,2017-08-31 17:12:56,87020,85.0,4,4782
3,001147e649a7b1afd577e873841632dd,364,2,424.32,0,2017-08-31 17:12:56,87020,85.0,4,4782
4,003e2af81ea3c72fbdd093f07724a4de,136,2,127.48,1,2018-04-16 20:59:52,2420,24.0,4,11450


In [10]:
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S')

In [11]:
df_cols=df.drop(['customer_unique_id','order_purchase_timestamp'], axis=1)
#numeric_cols.select_dtypes(include=[np.number]).columns
df_cols.columns

Index(['Recency', 'Frequency', 'MonetaryValue', 'Cluster',
       'customer_zip_code_prefix', 'price', 'review_score',
       'seller_zip_code_prefix'],
      dtype='object')

In [12]:
T_0 = min(df['order_purchase_timestamp']) 
T_0

Timestamp('2016-10-04 22:03:46')

In [13]:
n_months = 6
n_iterations = 4
T_n = T_0
T_list = []

for i in range(n_iterations):
    T_n = T_n + pd.DateOffset(months=n_months)
    T_list.append(T_n)
    print("T_{}: {}".format(i+1, T_n))

T_1: 2017-04-04 22:03:46
T_2: 2017-10-04 22:03:46
T_3: 2018-04-04 22:03:46
T_4: 2018-10-04 22:03:46


In [14]:
print("T_1:", T_list[0])
print("T_2:", T_list[1])

T_1: 2017-04-04 22:03:46
T_2: 2017-10-04 22:03:46


In [15]:
df_0 = df.loc[(df['order_purchase_timestamp'] >= T_0) & (df['order_purchase_timestamp'] <= T_list[0]), df_cols.columns]
df_0.columns

Index(['Recency', 'Frequency', 'MonetaryValue', 'Cluster',
       'customer_zip_code_prefix', 'price', 'review_score',
       'seller_zip_code_prefix'],
      dtype='object')

In [16]:
df_0.shape

(511, 8)

In [17]:
df_0.sample()

,Recency,Frequency,MonetaryValue,Cluster,customer_zip_code_prefix,price,review_score,seller_zip_code_prefix
8212,535,2,290.02,2,13212,129.0,5,13405


In [18]:
df_1 = df.loc[(df['order_purchase_timestamp'] >= T_0) & (df['order_purchase_timestamp'] <= T_list[1]), df_cols.columns]
df_1.columns

Index(['Recency', 'Frequency', 'MonetaryValue', 'Cluster',
       'customer_zip_code_prefix', 'price', 'review_score',
       'seller_zip_code_prefix'],
      dtype='object')

In [19]:
df_1.shape

(2759, 8)

In [20]:
df_1.sample(2)

,Recency,Frequency,MonetaryValue,Cluster,customer_zip_code_prefix,price,review_score,seller_zip_code_prefix
6872,538,2,145.8,1,6440,59.9,5,14940
696,432,2,236.4,1,8120,50.1,3,14940


In [21]:
X_0 = df_0.drop(['Cluster'], axis=1)
X_0.shape

(511, 7)

In [23]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import normalize
X_0_log_transformed = np.log1p(X_0)
X_0_normal = normalize(X_0_log_transformed)

In [25]:
from sklearn.cluster import KMeans
M0 = KMeans(n_clusters=3, random_state=42)
M0.fit(X_0_normal)
C_0 = M0.predict(X_0_normal)

In [26]:
C_0.shape

(511,)

In [27]:
X_1 = df_1.drop('Cluster', axis=1)
X_1.shape

(2759, 7)

In [28]:
X_1_log_transformed = np.log1p(X_1)
X_1_normal = normalize(X_1_log_transformed)

In [29]:
M1 = KMeans(n_clusters=3, random_state=0)
M1.fit(X_1_normal)
C_1 = M1.predict(X_1_normal)

In [30]:
C_1.shape

(2759,)

In [31]:
C1_init = M0.predict(X_1_normal)
C1_init.shape

(2759,)

In [32]:
X_0.reset_index(drop=True, inplace=True)
X_1.reset_index(drop=True, inplace=True)

On transfomre C_1 et C1_init en DataFrames puis réorganisons les indices :

In [34]:
C1_df = pd.DataFrame({'cluster': C_1})
C1_df.reset_index(drop=True, inplace=True)

In [36]:
C1_init_df = pd.DataFrame({'cluster': C1_init})
C1_init_df.reset_index(drop=True, inplace=True)

On détermine les clients communs entre X_0 et X_1

In [37]:
clients_T0_T1 = np.array(list(set(X_0.index).intersection(X_1.index)))
clients_T0_T1.shape

(511,)

On Extrait les clusters des clients communs à T_0 et T_1 et les transformons en numpay

In [38]:
C1_init_sorted = C1_init_df.loc[clients_T0_T1, 'cluster'].to_numpy()
C1_sorted = C1_df.loc[clients_T0_T1, 'cluster'].to_numpy()

In [39]:
# On compare les clusters
diff_clusters = sum(C1_init_sorted != C1_sorted)
print(f"Nombre de clients ayant changé de cluster : {diff_clusters}")

Nombre de clients ayant changé de cluster : 353


In [40]:
# Filtrer les données du dataframe X_1 pour ne garder que les données des clients qui ont changé de cluster
X_1_changed = X_1[X_1.index.isin(clients_T0_T1)]

# Ajouter les numéros de cluster à X_1 pour chaque client
X_1_changed['C1'] = C1_sorted

# Examiner les données de ces clients
X_1_changed.head(6)

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_12388\2914308182.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_1_changed['C1'] = C1_sorted


,Recency,Frequency,MonetaryValue,customer_zip_code_prefix,price,review_score,seller_zip_code_prefix,C1
0,335,2,93.70,11095,35.0,5,15013,0
1,335,2,93.70,11095,35.0,4,15013,0
2,364,2,424.32,87020,85.0,4,4782,2
3,364,2,424.32,87020,85.0,4,4782,2
4,527,2,163.56,3178,69.9,5,14940,0
5,527,2,163.56,3178,69.9,3,14940,0


In [41]:
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(C1_init_sorted, C1_sorted)
print(f"ARI entre les clusters à T0 et T1 : {ari}")

ARI entre les clusters à T0 et T1 : 0.8112408354280909


In [42]:
df_2 = df.loc[(df['order_purchase_timestamp'] >= T_0) & (df['order_purchase_timestamp'] <= T_list[2]), df_cols.columns]
df_2.shape

(6257, 8)

In [43]:
X_2 = df_2.drop('Cluster', axis=1)
X_2.shape

(6257, 7)

In [44]:
X_2_log_transformed = np.log1p(X_2)
X_2_normal= normalize(X_2_log_transformed)

In [45]:
M2 = KMeans(n_clusters=3, random_state=42)
M2.fit(X_2_normal)
C_2 = M2.predict(X_2_normal)

In [46]:
C2_init = M0.predict(X_2_normal)
C2_init.shape

(6257,)

In [47]:
X_2.reset_index(drop=True, inplace=True)
C2_df = pd.DataFrame({'cluster': C_2})
C2_df.reset_index(drop=True, inplace=True)

In [48]:
C2_init_df = pd.DataFrame({'cluster': C2_init})
C2_init_df.reset_index(drop=True, inplace=True)

In [50]:
#On détermine les clients communs à X_0 et X_2
clients_T0_T2 = np.array(list(set(X_0.index).intersection(X_2.index)))
clients_T0_T2.shape

(511,)

In [51]:
# On Extrait les clusters des clients communs à T_1 et T_2 et les transformons en numpay
C2_init_sorted = C2_init_df.loc[clients_T0_T2, 'cluster'].to_numpy()
C2_sorted = C2_df.loc[clients_T0_T2, 'cluster'].to_numpy()

In [52]:
# On compare les clusters
diff_clusters = sum(C2_init_sorted != C2_sorted)
print(f"Nombre de clients ayant changé de cluster : {diff_clusters}")

Nombre de clients ayant changé de cluster : 306


In [54]:
# Filtrer les données du dataframe X_1 pour ne garder que les données des clients qui ont changé de cluster
X_2_changed = X_2[X_2.index.isin(clients_T0_T2)]

# Ajouter les numéros de cluster à X_1 pour chaque client
X_2_changed['C2'] = C2_sorted

# Examiner les données de ces clients
X_2_changed.head()

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_12388\3395187778.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_2_changed['C2'] = C2_sorted


,Recency,Frequency,MonetaryValue,customer_zip_code_prefix,price,review_score,seller_zip_code_prefix,C2
0,335,2,93.70,11095,35.00,5,15013,1
1,335,2,93.70,11095,35.00,4,15013,1
2,364,2,424.32,87020,85.00,4,4782,2
3,364,2,424.32,87020,85.00,4,4782,2
4,233,2,297.52,63560,47.49,5,11701,2


In [55]:
ari = adjusted_rand_score(C2_init_sorted, C2_sorted)
print(f"ARI entre les clusters à T0 et T2 : {ari}")

ARI entre les clusters à T0 et T2 : 0.4498495976423019


Conclusion : la maintenance doit être faite tous les six mois.